In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt

from importlib import reload
import glob
from desitarget.targetmask import desi_mask, obsconditions

import mtl
import targets
import desitarget.mtl
import desitarget.targets

%matplotlib inline

In [2]:
# global truth file
truth = Table.read("targets/subset_truth_dr8_mtl_dark_gray_NGC.fits")

# This is the input to fiberassign
full_mtl = Table.read("strategy_A/targets/dark1_subset_dr8_mtl_dark_gray_northern_cap.fits")

# This is the results of fiberassign
fibertiles = glob.glob("strategy_A/fiberassign_gray/tile-*.fits")

# This is the final zcat after the run
zcat = Table.read("strategy_A/zcat/dark1_zcat.fits")

# This is the new mtl after using zcat
new_mtl = Table.read("strategy_A/targets/dark0_subset_dr8_mtl_dark_gray_northern_cap.fits")

In [3]:
set(zcat["SPECTYPE"])

{'GALAXY', 'QSO', 'STAR'}

In [4]:
print(len(truth), len(full_mtl), len(new_mtl), len(zcat), len(fibertiles))

4139902 4139902 4139902 565983 48


In [5]:
# select cases of DESI_TARGET!=0 that are also BGS_ANY or MWS_ANY
is_qso = (full_mtl['DESI_TARGET']&desi_mask.QSO)!=0
is_elg = (full_mtl['DESI_TARGET']&desi_mask.ELG)!=0
is_bgs_any = (full_mtl['DESI_TARGET']&desi_mask.BGS_ANY)!=0
is_mws_any = (full_mtl['DESI_TARGET']&desi_mask.MWS_ANY)!=0
is_qso_only = is_qso & (~is_elg) & (~is_bgs_any) & (~is_mws_any)
is_elg_only = is_elg & (~is_qso) & (~is_bgs_any) & (~is_mws_any)
is_qso_and_bgs = is_qso & is_bgs_any
is_qso_and_elg = is_qso & is_elg

print(np.count_nonzero(is_qso_only))
print(np.count_nonzero(is_qso_and_elg))
print(np.count_nonzero(is_bgs_any & is_qso))
print(np.count_nonzero(is_mws_any & is_qso))

165325
53853
27786
7054


In [6]:
mtl = reload(mtl)
targets = reload(targets)

In [7]:
full_mtl[is_qso_only][:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
163.335356587,-0.604994458379,35185748411941018,262148,0,0,3400,0.120315736953,4,27023,4,3400,1
163.383481929,-0.603631784248,35185748411941330,262148,0,0,3400,0.947191403352,4,27023,4,3400,1
163.404682212,-0.576845058547,35185748411941875,262148,0,0,3400,0.301269395122,4,27023,4,3400,1
163.484241573,-0.524829820853,35185748411942838,262148,0,0,3400,0.774462650836,4,27023,4,3400,1
163.258374217,-0.514782378641,35185748411942969,262148,0,0,3400,0.855551816096,4,27023,4,3400,1
163.327007644,-0.510185374596,35185748411943097,262148,0,0,3400,0.538996424223,4,27023,4,3400,1
163.322190355,-0.47157725339,35185748411943860,262148,0,0,3400,0.25551012992,4,27023,4,3400,1
163.416413221,-0.46517730371,35185748411943995,262148,0,0,3400,0.934074151249,4,27023,4,3400,1
163.289844513,-0.426915114372,35185748411944726,262148,0,0,3400,0.231879625891,4,27023,4,3400,1


In [8]:
is_qso_only_in_zcat = np.in1d(zcat['TARGETID'], full_mtl['TARGETID'][is_qso_only])
print(set(zcat[is_qso_only_in_zcat]['Z']))

{2.0, 3.0}


In [9]:
is_qso_only_in_zcat = np.in1d(zcat['TARGETID'], full_mtl['TARGETID'][is_qso_only])
print(set(zcat[is_qso_only_in_zcat]['Z']))

{2.0, 3.0}


In [10]:
full_mtl[is_qso_and_bgs][:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
163.409892939,-0.467889237659,35185748411943948,3458764513820803076,262148,1280,3400,0.443406021263,4,27023,4,3400,7
163.416488992,-0.406207875064,35185748411945199,1152921504607109124,262148,0,3400,0.468286447957,4,27023,4,3400,5
163.384626236,-0.389135752321,35185748411945548,1152921504607109124,262148,0,3400,0.00996304762306,4,27023,4,3400,5
163.948338835,-2.51105531989,35185700101949546,3458764513820803076,262148,1280,3400,0.234914107368,4,27014,4,3400,7
163.295087239,-4.85720192471,35185645802488093,1152921504607109124,262148,0,3400,0.741749247155,4,26922,4,3400,5
163.362903087,-4.81107962452,35185645802489082,3458764513820803076,262148,1280,3400,0.505486726042,4,26922,4,3400,7
163.217154151,-4.75728790347,35185645802490243,1152921504607109124,262148,0,3400,0.798630463715,4,26922,4,3400,5
163.202027582,-4.24031799567,35185657848531171,1152921504607109124,262148,0,3400,0.231919722975,4,27008,4,3400,5
163.18158437,-4.23196332021,35185657848531322,1152921504607109124,262148,0,3400,0.175279856885,4,27008,4,3400,5


In [11]:
mtl = reload(mtl)
targets = reload(targets)
new_code_mtl = mtl.make_mtl(full_mtl, zcat, bright=False)

is_qso_only_aa = np.in1d(new_code_mtl['TARGETID'], full_mtl['TARGETID'][is_qso_only])
print(set(new_code_mtl[is_qso_only_aa]['NUMOBS_MORE']))

is_qso_and_bgs_aa = np.in1d(new_code_mtl['TARGETID'], full_mtl['TARGETID'][is_qso_and_bgs])
print(set(new_code_mtl[is_qso_and_bgs_aa]['NUMOBS_MORE']))

is_qso_and_elg_aa = np.in1d(new_code_mtl['TARGETID'], full_mtl['TARGETID'][is_qso_and_elg])
print(set(new_code_mtl[is_qso_and_elg_aa]['NUMOBS_MORE']))

is_elg_only_aa = np.in1d(new_code_mtl['TARGETID'], full_mtl['TARGETID'][is_elg_only])
print(set(new_code_mtl[is_elg_only_aa]['NUMOBS_MORE']))

INFO:mtl.py:125:make_mtl: 453717 of 4139902 targets have priority zero, setting N_obs=0.
{0, 1, 2, 3, 4}
{0, 1, 2, 3, 4}
{0, 1, 2, 3, 4}
{0, 1}


In [12]:
old_code_mtl = desitarget.mtl.make_mtl(full_mtl, zcat)

is_qso_only_aa = np.in1d(old_code_mtl['TARGETID'], full_mtl['TARGETID'][is_qso_only])
print(set(old_code_mtl[is_qso_only_aa]['NUMOBS_MORE']))

is_qso_and_bgs_aa = np.in1d(old_code_mtl['TARGETID'], full_mtl['TARGETID'][is_qso_and_bgs])
print(set(old_code_mtl[is_qso_and_bgs_aa]['NUMOBS_MORE']))

is_qso_and_elg_aa = np.in1d(old_code_mtl['TARGETID'], full_mtl['TARGETID'][is_qso_and_elg])
print(set(old_code_mtl[is_qso_and_elg_aa]['NUMOBS_MORE']))

is_elg_only_aa = np.in1d(old_code_mtl['TARGETID'], full_mtl['TARGETID'][is_elg_only])
print(set(old_code_mtl[is_elg_only_aa]['NUMOBS_MORE']))

INFO:mtl.py:119:make_mtl: 425984 of 4139902 targets have priority zero, setting N_obs=0.
{0, 1, 2, 3, 4}
{1, 4}
{0, 1, 2, 3, 4}
{0, 1}
